<a href="https://colab.research.google.com/github/Luis-Rojas-H/999911/blob/master/Conteo_de_dinero_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os                         # <--- acceso a ficheros
import numpy as np                # <---- op. matemáticas
from skimage.io import imread     # <---- leer imágenes
import matplotlib.pyplot as plt   # <---- mostrar las imágenes
from skimage.filters import threshold_otsu, threshold_local # <-- umbralizar
from skimage.measure import label, regionprops              # <-- segmentar por regiones
from skimage.transform import rescale                       # <-- reducir tam. imagen
from skimage.exposure  import equalize_hist                 # <-- ecualizar
from termcolor import colored                               # <-- dar color a regiones

### Obtener los datos

In [ ]:
!gdown 1trBZlOCo9O4AV_syWEL46cfHoagTmJ-p
!unzip img.zip
imagenes = os.listdir('img')

### Preprocesar la imágen, umbralizar y etiquetar componentes conectados

In [ ]:
from scipy import ndimage
from skimage.segmentation import watershed
from skimage.feature import peak_local_max

In [ ]:
#@title Default title text { run: "auto" }
im_id = 0 #@param {type:"slider", min:0, max:8, step:1}

im_filename = imagenes[im_id]
im_path = os.path.join('img',im_filename)
im = imread(im_path)
im_grayscale = im[:,:,0]
im_grayscale -= im_grayscale.min()
im_grayscale = im_grayscale/im_grayscale.max()
th = threshold_otsu(im_grayscale)
im_th = im_grayscale>th
im_labeled = label(im_th)

#Mostrar imagenes
plt.figure(figsize=(20,5))
plt.subplot(1,4,1)
plt.title("Imagen de entrada")
plt.imshow(im)
plt.subplot(1,4,2)
plt.title("Imagen Umbralizada")
plt.imshow(im_th)
plt.subplot(1,4,3)
plt.title("Imagen watershed")
distance = ndimage.distance_transform_edt(im_th)
local_maxi = peak_local_max(distance, footprint=np.ones((3, 3)),labels=im_th)

# Create a markers array with the same shape as im_th
markers = np.zeros_like(im_th)
# Set the markers based on the coordinates in local_maxi
markers[local_maxi[:, 0], local_maxi[:, 1]] = 1
markers = ndimage.label(markers)[0]
# Now markers has the same shape as im_th and contains the correct labels

labels = watershed(-distance, markers, mask=im_th)
plt.imshow(labels, cmap='nipy_spectral')
plt.subplot(1,4,4)
plt.title("Componentes conectados")
plt.imshow(im_labeled, cmap='nipy_spectral')

In [ ]:
from skimage.measure import regionprops

a_min, a_max = np.pi*80**2, np.pi*150**2


for r in regionprops(im_labeled):

  print("Area: {}".format(r.area))
  if (r.area < a_min or r.area > a_max):
    print("No pasa el filtro de area")
  else:
    print("Procesando la region!")
    print("Excentricidad de la region es {}".format(r.eccentricity))
    if r.eccentricity < 0.5:
      print("Paso el filtro de circularidad")
      min_row, min_col, max_row, max_col = r.bbox
      plt.figure()
      plt.imshow(im[min_row:max_row, min_col:max_col])
      plt.show()
      print(im[min_row:max_row, min_col:max_col].mean(axis=0).mean(axis=0))


  print("=============")

In [ ]:
from skimage.measure import regionprops

a_min, a_max = np.pi*80**2, np.pi*150**2

monto = 0

for r in regionprops(im_labeled):

  # print("Area: {}".format(r.area))
  if (r.area < a_min or r.area > a_max):
    # print("No pasa el filtro de area")
    pass
  else:
    # print("Procesando la region!")
    # print("Excentricidad de la region es {}".format(r.eccentricity))
    if r.eccentricity < 0.5:
      print(r.area)
      print("Excentricidad de la region es {}".format(r.eccentricity))
      # print("Paso el filtro de circularidad")
      min_row, min_col, max_row, max_col = r.bbox
      plt.figure()
      plt.imshow(im[min_row:max_row, min_col:max_col])
      plt.show()
      print(im[min_row:max_row, min_col:max_col].mean(axis=0).mean(axis=0))

      if (r.area >= 26000 and r.area <= 30000):
        monto += 0.10
      elif (r.area >= 30000 and r.area <= 33000):
        monto += 0.50
      elif (r.area >= 42000):
        monto += 1


  print("=============")
print("Monto total:", monto)